In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/dataset/smsspamcollection.tsv', sep = '\t')

In [3]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


# Vectorize

---

---


` # Now we're going to take the raw text in 'message' column and vectorize it.`

In [4]:
# Firstly check for any missing values in the columns
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [5]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64



```
# Now let's split the data into a training test and test set
```



In [6]:
from sklearn.model_selection import train_test_split

In [7]:
# Let's only extract the message (raw text) as the feature itself
X = df['message']
y = df['label']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



```
# Let's do Count Vectorization now
```



In [9]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()



```
# There are two ways to vectorize:
1. Fit vectorizer to the data (build vocab, count the number of words)
2. Transform the original text message into a Vector
```



In [10]:
# count_vect.fit(X_train)
# X_train_counts = count_vect.transform(X_train)

# The above two steps can be shortened into one step:
X_train_counts = count_vect.fit_transform(X_train)

In [11]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [12]:
X_train_counts.shape

(3733, 7082)

In [13]:
X_train_counts.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])



```
# Let's create a TF-IDF Vectorizer
```



In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

In [15]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [16]:
X_train_tfidf.shape

# It is stil the same shape, but it's no longer just the count. 
# We've now taken the term frequency and multiplied it with it's inverse document frequency 

(3733, 7082)



```
# The above two steps can also be done in one single step in Scikit-Learn.
```



In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [18]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [19]:
X_train_tfidf.shape

(3733, 7082)



```
# Let's train a classifier now
```



In [20]:
from sklearn.svm import LinearSVC
clf = LinearSVC().fit(X_train_tfidf, y_train)

In [21]:
clf

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [22]:
# We can everything mentioned above in one single line using Pipeline in scikit learn.
from sklearn.pipeline import Pipeline
test_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())]).fit(X_train, y_train)

In [23]:
test_clf

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [24]:
predictions = test_clf.predict(X_test)



```
# Creating Confusion Matrix
```



In [25]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

In [26]:
print(confusion_matrix(y_test, predictions))

[[1586    7]
 [  12  234]]


In [27]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [28]:
print(metrics.accuracy_score(y_test, predictions))

0.989668297988037




```
# Predicting on a new message string
```



In [29]:
test_clf.predict(["Hi, this is Rudra!"])

array(['ham'], dtype=object)

In [30]:
test_clf.predict(["You've won $2Bn in a lottery hamper! TEXT won to 12344. Congratulations***"])

array(['spam'], dtype=object)